In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
main_url = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXII#'

In [3]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [4]:
driver.get(main_url)

In [12]:
import math
def load_all_pages(driver, partido):
    time.sleep(5)
    if any(re.findall('No hay intervenciones',driver.find_element_by_id('navega').text)):
        print('no intervenciones')
    else:
        try:            
            number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            while not any(number_of_intervenciones):
                time.sleep(1)
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            else:
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))

            number_of_loops = math.ceil(int(number_of_intervenciones)/20)

            if number_of_loops == 1:
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))
                else:
                    return(obtain_metadata(driver, partido))
            else:
                all_data = []
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))

                else:
                    all_data.append(obtain_metadata(driver, partido))
                    r = 20
                    for i in range(1,number_of_loops):
                        time.sleep(3)
                        driver.find_element_by_id('next').click()
                        elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                        e = int(elements_to_scrape) - r
                        entries_found = len(driver.find_elements_by_class_name('NoI'))
                        while entries_found < e:
                            time.sleep(1)
                            entries_found = len(driver.find_elements_by_class_name('NoI'))
                        else:
                            all_data.append(obtain_metadata(driver, partido))
                        r += 20
                    return(pd.concat(all_data))
        except:
            time.sleep(5)
            number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            while not any(number_of_intervenciones):
                time.sleep(1)
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))
            else:
                number_of_intervenciones = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text))

            number_of_loops = math.ceil(int(number_of_intervenciones)/20)

            if number_of_loops == 1:
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))
                else:
                    return(obtain_metadata(driver, partido))
            else:
                all_data = []
                entries_found = len(driver.find_elements_by_class_name('NoI'))
                elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                while entries_found < int(elements_to_scrape):
                    time.sleep(1)
                    entries_found = len(driver.find_elements_by_class_name('NoI'))

                else:
                    all_data.append(obtain_metadata(driver, partido))
                    r = 20
                    for i in range(1,number_of_loops):
                        time.sleep(3)
                        driver.find_element_by_id('next').click()
                        elements_to_scrape = max(re.findall('[0-9]+', driver.find_element_by_id('navega').text.split(',')[0]))

                        e = int(elements_to_scrape) - r
                        entries_found = len(driver.find_elements_by_class_name('NoI'))
                        while entries_found < e:
                            time.sleep(1)
                            entries_found = len(driver.find_elements_by_class_name('NoI'))
                        else:
                            all_data.append(obtain_metadata(driver, partido))
                        r += 20
                    return(pd.concat(all_data))

In [6]:
def obtain_metadata(driver, party):
    all_data = []
    
    years = []
    tipo_sesion = []
    periodo = []
    fecha = []
    diario = []
    tema = []
    link_speech = []
    
    for i in driver.find_element_by_id('output').find_elements_by_class_name('YearD'):
        years.append(i.text)
    
    for i in driver.find_element_by_id('output').find_elements_by_class_name('FechaD'):
        fecha.append(i.text)
        
    for i in driver.find_element_by_id('output').find_elements_by_class_name('TemaI'):
        tema.append(i.text)
        link_speech.append(i.find_element_by_css_selector('a').get_attribute('href'))
    
    df = pd.DataFrame({'year_session':years, 
                       'fecha': fecha, 
                       'tema':tema,
                       'link_speech':link_speech
                        })
    df['partido'] = party
    df['name'] = driver.find_element_by_id('Escudo').text
    return(df)

In [7]:
def obtain_allData(driver, partido):
    names_deputies_party = []
    partido_list = []
    for i in driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option'):
        names_deputies_party.append(i.text)
        partido_list.append(partido)
        
    all_d = []
    for i, e in tqdm(zip(driver.find_element_by_id('selectDipu').find_elements_by_css_selector('option'), names_deputies_party), total = len(names_deputies_party)):
        i.click()
        time.sleep(5)
        all_d.append(load_all_pages(driver, partido))
    return(all_d)

## Legislatura 64

In [551]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [552]:
### Loop to all Parties
url_legislatura64 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid3.xql?legis=LXIV'
driver.get(url_legislatura64)

In [553]:
num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

In [562]:
name_parties[4:]

['Partido del Trabajo',
 'Movimiento Ciudadano',
 'Partido de la Revolución Democrática',
 'Partido Verde Ecologista de México',
 'Sin Partido']

In [ ]:
all_dataLegislatura = []

In [563]:
for x, y in tqdm(zip(range(num_parties)[4:], name_parties[4:]), total = len(name_parties[4:])):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura64)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
        
    all_dataLegislatura.append(obtain_allData(driver, y))
    driver.close()

  0%|          | 0/5 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 20%|██        | 1/5 [06:04<24:17, 364.44s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 40%|████      | 2/5 [10:13<16:29, 329.75s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 60%|██████    | 3/5 [14:22<10:10, 305.48s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 80%|████████  | 4/5 [16:21<04:09, 249.64s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 5/5 [16:36<00:00, 199.31s/it]


In [575]:
all_d_faltten = []
for i in all_dataLegislatura:
    all_d_faltten.append(pd.concat(i))

In [576]:
all_d_faltten = pd.concat(all_d_faltten).reset_index().drop(['index'], axis = 1)

In [577]:
len(all_d_faltten)

8332

In [578]:
all_d_faltten.to_csv('./../../data/diputados/floor-speech/data_links_speeches64.csv', encoding = 'utf8')

### Legislatura 63

In [592]:
### Loop to all Parties
url_legislatura63 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXIII#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura63)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura63 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura63)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()

    all_dataLegislatura63.append(obtain_allData(driver, y))
    driver.close()

all_d_faltten63 = []
for i in all_dataLegislatura63:
    all_d_faltten63.append(pd.concat(i))

all_d_faltten63 = pd.concat(all_d_faltten63).reset_index().drop(['index'], axis = 1)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


  0%|          | 0/9 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 11%|█         | 1/9 [56:02<7:28:17, 3362.17s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 22%|██▏       | 2/9 [1:17:06<5:18:49, 2732.77s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 33%|███▎      | 3/9 [1:31:11<3:36:39, 2166.57s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


...

no intervenciones



 67%|██████▋   | 6/9 [2:04:08<58:09, 1163.33s/it]  [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache

 78%|███████▊  | 7/9 [2:09:29<30:21, 910.61s/it] [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 89%|████████▉ | 8/9 [2:13:54<11:56, 716.87s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 9/9 [2:14:24<00:00, 896.02s/it]


In [594]:
all_d_faltten63.to_csv('./../../data/diputados/floor-speech/data_links_speeches63.csv', encoding = 'utf8')

### Legislatura 62

In [16]:
### Loop to all Parties
url_legislatura62 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXII#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura62)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura62 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura62)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        all_dataLegislatura62.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura62.append(obtain_allData(driver, y))
    driver.close()

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


  0%|          | 0/7 [00:00<?, ?it/s][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 71%|███████▏  | 155/217 [27:31<10:47, 10.44s/it]


 95%|█████████▌| 207/217 [36:45<01:42, 10.20s/it]

no intervenciones


...


 43%|████▎     | 3/7 [1:31:48<2:11:37, 1974.36s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 57%|█████▋    | 4/7 [1:40:04<1:16:33, 1531.13s/it][WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 71%|███████▏  | 5/7 [1:49:08<41:09, 1234.83s/it]  [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



 86%|████████▌ | 6/7 [1:56:02<16:28, 988.47s/it] [WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache



100%|██████████| 7/7 [2:00:29<00:00, 1032.75s/it]


In [17]:
all_d_faltten62 = []
for i in all_dataLegislatura62:
    all_d_faltten62.append(pd.concat(i))

all_d_faltten62 = pd.concat(all_d_faltten62).reset_index().drop(['index'], axis = 1)

In [18]:
all_d_faltten62.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/01-floor_speech/00-scrape_speeches/links_speeches62.csv', encoding = 'utf8')

### Legislatura 61

In [ ]:
# Loop to all Parties
url_legislatura61 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXI#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura61)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura61 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura61)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        all_dataLegislatura61.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura61.append(obtain_allData(driver, y))
    driver.close()

In [ ]:
all_d_faltten61 = []
for i in all_dataLegislatura61:
    all_d_faltten61.append(pd.concat(i))

all_d_faltten61 = pd.concat(all_d_faltten61).reset_index().drop(['index'], axis = 1)

In [18]:
all_d_faltten61.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/01-floor_speech/00-scrape_speeches/links_speeches61.csv', encoding = 'utf8')

### Legislatura 60

In [ ]:
# Loop to all Parties
url_legislatura61 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LX#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura61)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura61 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura61)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        all_dataLegislatura61.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura61.append(obtain_allData(driver, y))
    driver.close()

In [17]:
all_d_faltten61 = []
for i in all_dataLegislatura61:
    all_d_faltten61.append(pd.concat(i))

all_d_faltten61 = pd.concat(all_d_faltten61).reset_index().drop(['index'], axis = 1)

In [18]:
all_d_faltten61.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/01-floor_speech/00-scrape_speeches/links_speeches61.csv', encoding = 'utf8')

In [ ]:
### Legislatura 61

# Loop to all Parties
url_legislatura61 = 'http://cronica.diputados.gob.mx:8080/exist/siid2/siid2.xql?legis=LXI#'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url_legislatura61)

num_parties = len(driver.find_elements_by_class_name('Grupos-Menu-subItem'))
name_parties = []
for i in driver.find_elements_by_class_name('Grupos-Menu-subItem'):
    if re.sub('\n', ' ', i.text) not in name_parties:
        name_parties.append(re.sub('\n', ' ', i.text))
    else:
        continue

all_dataLegislatura61 = []
for x, y in tqdm(zip(range(num_parties), name_parties), total = len(name_parties)):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url_legislatura61)
    driver.find_elements_by_class_name('Grupos-Menu-subItem')[int(x)].click()
    try:
        all_dataLegislatura61.append(obtain_allData(driver, y))
    except:
        time.sleep(10)
        all_dataLegislatura61.append(obtain_allData(driver, y))
    driver.close()

all_d_faltten61 = []
for i in all_dataLegislatura61:
    all_d_faltten61.append(pd.concat(i))

all_d_faltten61 = pd.concat(all_d_faltten61).reset_index().drop(['index'], axis = 1)

all_d_faltten61.to_csv('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/floor-speechMX/data/01-floor_speech/00-scrape_speeches/links_speeches61.csv', encoding = 'utf8')